## Import

In [ ]:
# %pip install pyarrow
# %pip install xgboost

In [1]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

## Data Load

In [2]:
# 데이터 분할(폴더) 구분
data_splits = ["train", "test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

# 2018년 7월부터 12월까지의 월 리스트
months = ['07', '08', '09', '10', '11', '12']

for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]
        
        for month in months:
            # 파일명 형식: 2018{month}_{split}_{suffix}.parquet
            file_path = f"../data/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
            print(file_path)
            # 변수명 형식: {var_prefix}_{split}_{month}
            variable_name = f"{var_prefix}_{split}_{month}"
            globals()[variable_name] = pd.read_parquet(file_path)
            print(f"{variable_name} is loaded from {file_path}")

gc.collect()

../data/train/1.회원정보/201807_train_회원정보.parquet
customer_train_07 is loaded from ../data/train/1.회원정보/201807_train_회원정보.parquet
../data/train/1.회원정보/201808_train_회원정보.parquet
customer_train_08 is loaded from ../data/train/1.회원정보/201808_train_회원정보.parquet
../data/train/1.회원정보/201809_train_회원정보.parquet
customer_train_09 is loaded from ../data/train/1.회원정보/201809_train_회원정보.parquet
../data/train/1.회원정보/201810_train_회원정보.parquet
customer_train_10 is loaded from ../data/train/1.회원정보/201810_train_회원정보.parquet
../data/train/1.회원정보/201811_train_회원정보.parquet
customer_train_11 is loaded from ../data/train/1.회원정보/201811_train_회원정보.parquet
../data/train/1.회원정보/201812_train_회원정보.parquet
customer_train_12 is loaded from ../data/train/1.회원정보/201812_train_회원정보.parquet
../data/train/2.신용정보/201807_train_신용정보.parquet
credit_train_07 is loaded from ../data/train/2.신용정보/201807_train_신용정보.parquet
../data/train/2.신용정보/201808_train_신용정보.parquet
credit_train_08 is loaded from ../data/train/2.신용정보/201808_train_신

0

## Data Preprocessing

### 1. Concat & Merge

In [3]:
# 데이터 유형별 설정 
info_categories = ["customer", "credit", "sales", "billing", "balance", "channel", "marketing", "performance"]

# 월 설정
months = ['08', '09', '10', '11', '12']

In [4]:
#### Train ####

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train_{month}"] for month in months]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")


customer_train_df = train_dfs["customer_train_df"]
credit_train_df   = train_dfs["credit_train_df"]
sales_train_df    = train_dfs["sales_train_df"]
billing_train_df  = train_dfs["billing_train_df"]
balance_train_df  = train_dfs["balance_train_df"]
channel_train_df  = train_dfs["channel_train_df"]
marketing_train_df= train_dfs["marketing_train_df"]
performance_train_df = train_dfs["performance_train_df"]

gc.collect()

customer_train_df is created with shape: (2000000, 78)
credit_train_df is created with shape: (2000000, 42)
sales_train_df is created with shape: (2000000, 406)
billing_train_df is created with shape: (2000000, 46)
balance_train_df is created with shape: (2000000, 82)
channel_train_df is created with shape: (2000000, 105)
marketing_train_df is created with shape: (2000000, 64)
performance_train_df is created with shape: (2000000, 49)


0

In [5]:
#### Test ####

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test_{month}"] for month in months]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")


customer_test_df = test_dfs["customer_test_df"]
credit_test_df   = test_dfs["credit_test_df"]
sales_test_df    = test_dfs["sales_test_df"]
billing_test_df  = test_dfs["billing_test_df"]
balance_test_df  = test_dfs["balance_test_df"]
channel_test_df  = test_dfs["channel_test_df"]
marketing_test_df= test_dfs["marketing_test_df"]
performance_test_df = test_dfs["performance_test_df"]

gc.collect()

customer_test_df is created with shape: (500000, 77)
credit_test_df is created with shape: (500000, 42)
sales_test_df is created with shape: (500000, 406)
billing_test_df is created with shape: (500000, 46)
balance_test_df is created with shape: (500000, 82)
channel_test_df is created with shape: (500000, 105)
marketing_test_df is created with shape: (500000, 64)
performance_test_df is created with shape: (500000, 49)


0

In [6]:
#### Train ####

train_df = customer_train_df.merge(credit_train_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: train_step1, shape:", train_df.shape)
del customer_train_df, credit_train_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_train_df",    "Step2"),
    ("billing_train_df",  "Step3"),
    ("balance_train_df",  "Step4"),
    ("channel_train_df",  "Step5"),
    ("marketing_train_df","Step6"),
    ("performance_train_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    train_df = train_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: train_{step}, shape:", train_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: train_step1, shape: (2000000, 118)
Step2 저장 완료: train_Step2, shape: (2000000, 522)
Step3 저장 완료: train_Step3, shape: (2000000, 566)
Step4 저장 완료: train_Step4, shape: (2000000, 646)
Step5 저장 완료: train_Step5, shape: (2000000, 749)
Step6 저장 완료: train_Step6, shape: (2000000, 811)
최종 저장 완료: train_최종, shape: (2000000, 858)


In [7]:
#### Test ####

test_df = customer_test_df.merge(credit_test_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: test_step1, shape:", test_df.shape)
del customer_test_df, credit_test_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_test_df",    "Step2"),
    ("billing_test_df",  "Step3"),
    ("balance_test_df",  "Step4"),
    ("channel_test_df",  "Step5"),
    ("marketing_test_df","Step6"),
    ("performance_test_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    test_df = test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: test_{step}, shape:", test_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: test_step1, shape: (500000, 117)
Step2 저장 완료: test_Step2, shape: (500000, 521)
Step3 저장 완료: test_Step3, shape: (500000, 565)
Step4 저장 완료: test_Step4, shape: (500000, 645)
Step5 저장 완료: test_Step5, shape: (500000, 748)
Step6 저장 완료: test_Step6, shape: (500000, 810)
최종 저장 완료: test_최종, shape: (500000, 857)


### 데이터 확인(회원정보)

In [8]:
exclude_columns = [
    # 공통적으로 모든 시트에 포함된 식별자/시계열 변수
    '기준년월',

    # 1.회원 정보
    '입회일자_신용', '최종카드발급일자', '가입통신회사코드', '거주시도명', '직장시도명',

    # 2.신용 정보
    'RV신청일자', 'RV전환가능여부', '일시불ONLY전환가능여부',

    # 3.승인매출 정보
    '최종이용일자_기본', '최종이용일자_신판', '최종이용일자_CA',
    '최종이용일자_체크', '최종이용일자_일시불', '최종이용일자_할부', '최종이용일자_카드론',

    # 4.청구입금 정보
    '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법',

    # 5.잔액 정보
    '연체일자_B0M', '연체일수_B1M', '연체일수_B2M', '연체일수_최근', '최종연체회차', '최종연체개월수_R15M',

    # 6.채널 정보
    'OS구분코드', '방문후경과월_앱_R6M', '방문후경과월_PC_R6M', '방문후경과월_모바일웹_R6M',
    '인입후경과월_ARS', '인입후경과월_IB_R6M', '인입불만후경과월_IB_R6M',

    # 7.마케팅 정보
    # → 시계열 정보가 없어서 제외 항목 없음 (단, 필요에 따라 조정 가능)

    # 8.성과 정보
    # → 모두 파생지표로서 유용할 가능성이 높아 전체 사용 가능
]

In [9]:
print(test_df.info())
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Columns: 857 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float64(61), int64(747), object(49)
memory usage: 3.2+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 858 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float64(61), int64(747), object(50)
memory usage: 12.8+ GB
None


In [10]:
# 1. 데이터 타입 다운캐스트 함수
def downcast_dtypes(df):
    float_cols = df.select_dtypes(include='float64').columns
    int_cols = df.select_dtypes(include='int64').columns

    for col in float_cols:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in int_cols:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    
    return df

# 2. 제외할 변수 목록
# exclude_columns = [
#     'ID', '기준년월',
#     '입회일자_신용', '최종카드발급일자', '가입통신회사코드', '거주시도명', '직장시도명',
#     'RV신청일자', 'RV전환가능여부', '일시불ONLY전환가능여부',
#     '최종이용일자_기본', '최종이용일자_신판', '최종이용일자_CA',
#     '최종이용일자_체크', '최종이용일자_일시불', '최종이용일자_할부', '최종이용일자_카드론',
#     '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법',
#     '연체일자_B0M', '연체일수_B1M', '연체일수_B2M', '연체일수_최근', '최종연체회차', '최종연체개월수_R15M',
#     'OS구분코드', '방문후경과월_앱_R6M', '방문후경과월_PC_R6M', '방문후경과월_모바일웹_R6M',
#     '인입후경과월_ARS', '인입후경과월_IB_R6M', '인입불만후경과월_IB_R6M'
# ]

# 3. 적용 순서
# 먼저 train_df와 test_df를 불러온 후에 아래 적용
train_df = downcast_dtypes(train_df)
test_df = downcast_dtypes(test_df)

for col in exclude_columns:
    if col in train_df.columns:
        train_df.drop(columns=[col], inplace=True)
    if col in test_df.columns:
        test_df.drop(columns=[col], inplace=True)

print("✅ 최종 train_df shape:", train_df.shape)
print("✅ 최종 test_df shape:", test_df.shape)


✅ 최종 train_df shape: (2000000, 824)
✅ 최종 test_df shape: (500000, 823)


In [11]:
# 결측치 수 기준 정렬해서 DataFrame으로 보기
na_df = train_df.isnull().sum().reset_index()
na_df.columns = ['column', 'na_count']
na_df = na_df[na_df['na_count'] > 0].sort_values(by='na_count', ascending=False)
na_df['na_ratio'] = na_df['na_count'] / len(train_df)

# 전체 출력
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(na_df)

,column,na_count,na_ratio
283,_3순위여유업종,1986980,0.993490
281,_2순위여유업종,1935294,0.967647
289,_3순위납부업종,1925705,0.962853
277,_3순위교통업종,1709613,0.854807
287,_2순위납부업종,1697411,0.848706
279,_1순위여유업종,1689398,0.844699
370,최종카드론_대출일자,1656940,0.828470
368,최종카드론_신청경로코드,1631581,0.815790
367,최종카드론_금융상환방식코드,1631544,0.815772
275,_2순위교통업종,1386841,0.693420


In [12]:
# 1. 결측 비율 계산 (train 기준)
na_ratio = train_df.isnull().mean()

# 2. 결측 비율이 50% 이상인 컬럼만 추출
cols_to_drop = na_ratio[na_ratio >= 0.5].index.tolist()

# 3. 컬럼 제거 (메모리 절약 위해 한 컬럼씩)
for col in cols_to_drop:
    if col in train_df.columns:
        train_df.drop(columns=[col], inplace=True)
    if col in test_df.columns:
        test_df.drop(columns=[col], inplace=True)

# 4. 나머지 결측값은 0으로 채움
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

# 5. 결과 확인
print("✅ 최종 train_df shape:", train_df.shape)
print("✅ 최종 test_df shape:", test_df.shape)
print("🗑 제거된 컬럼 수:", len(cols_to_drop))


✅ 최종 train_df shape: (2000000, 812)
✅ 최종 test_df shape: (500000, 811)
🗑 제거된 컬럼 수: 12


### 2. Encoding

In [13]:
from sklearn.preprocessing import LabelEncoder

# 1. 타깃과 피처 분리
target_col = "Segment"
feature_cols = [col for col in train_df.columns if col not in ["ID", "기준년월", target_col]]

X = train_df[feature_cols].copy()
y = train_df[target_col].copy()

# ✅ ID는 따로 백업!
X_test_id = test_df["ID"].copy()
X_test = test_df[feature_cols].copy()

# 2. 범주형 변수 라벨 인코딩
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

encoders = {}
for col in categorical_features:
    X[col] = X[col].fillna("missing").astype(str)
    X_test[col] = X_test[col].fillna("missing").astype(str)

    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    encoders[col] = le

    # test에 없는 클래스 보완
    unseen_labels = set(X_test[col]) - set(le.classes_)
    if unseen_labels:
        le.classes_ = np.append(le.classes_, list(unseen_labels))
    X_test[col] = le.transform(X_test[col])

# 3. 수치형 결측값 처리
X.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

# 4. 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

# 5. ID 복원
X_test["ID"] = X_test_id

# ✅ 최종 확인
print("✅ X shape:", X.shape)
print("✅ X_test shape:", X_test.shape)
print("✅ y_encoded shape:", y_encoded.shape)
print("✅ 인코딩 완료된 범주형 변수 수:", len(categorical_features))


✅ X shape: (2000000, 810)
✅ X_test shape: (500000, 811)
✅ y_encoded shape: (2000000,)
✅ 인코딩 완료된 범주형 변수 수: 29


In [14]:
print(X.info())
print(X_test.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 810 entries, 남녀구분코드 to 혜택수혜율_B0M
dtypes: float32(56), int16(148), int32(134), int64(29), int8(443)
memory usage: 3.2 GB
None
Index(['남녀구분코드', '연령', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용',
       '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회경과개월수_신용', '회원여부_연체',
       ...
       '변동률_할부평잔', '변동률_CA평잔', '변동률_RVCA평잔', '변동률_카드론평잔', '변동률_잔액_B1M',
       '변동률_잔액_일시불_B1M', '변동률_잔액_CA_B1M', '혜택수혜율_R3M', '혜택수혜율_B0M', 'ID'],
      dtype='object', length=811)


### Train

In [15]:
# print(X.isnull().sum())
# # print(y_encoded.isnull().sum())
# print(y_encoded)
# print(X)
# for col in X.columns:
#     X[col].fillna(0, inplace=True)

In [16]:
print(X.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 810 entries, 남녀구분코드 to 혜택수혜율_B0M
dtypes: float32(56), int16(148), int32(134), int64(29), int8(443)
memory usage: 3.2 GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Columns: 811 entries, ID to 혜택수혜율_B0M
dtypes: float32(56), int16(154), int32(126), int8(445), object(30)
memory usage: 820.6+ MB
None


In [17]:
model = xgb.XGBClassifier(tree_method='hist', random_state = 42)  # CPU 기반 학습
model.fit(X, y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

### Predict

In [22]:
X_test.drop(columns=['ID'],inplace=True)

In [21]:
X_test.info()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Columns: 811 entries, 남녀구분코드 to ID
dtypes: float32(56), int16(154), int32(126), int64(29), int8(445), object(1)
memory usage: 820.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 810 entries, 남녀구분코드 to 혜택수혜율_B0M
dtypes: float32(56), int16(148), int32(134), int64(29), int8(443)
memory usage: 3.2 GB


In [23]:
# row-level 예측 수행
y_test_pred = model.predict(X_test)
# 예측 결과를 변환
y_test_pred_labels = le_target.inverse_transform(y_test_pred)

# row 단위 예측 결과를 test_data에 추가
test_data = test_df.copy()  # 원본 유지
test_data["pred_label"] = y_test_pred_labels

### Submission

In [24]:
submission = test_data.groupby("ID")["pred_label"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

submission.columns = ["ID", "Segment"]
submission.to_csv('../submit/0326_2.csv',index=False)

In [ ]:
print(test_data.head())

NameError: name 'test_data' is not defined